# Example: compute the center-of-mass and the orientaiton of individual cations in a ionic liquid

* a simulation of a thin ionic liquid film was performed
* the ```lammps``` data was saved in ````result_atoms.data````, which contains the topology information of the system
* a short trajectory was saved in ```result_atoms.lammpstrj```
* note that in ```result_atoms.lammpstrj```, atom properties should be save in the order of ```id type x y z```, so that it can be read by ```mdanalysis```

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
pd.options.display.max_rows = 6
pd.options.display.max_columns = None
pd.options.display.width = 100
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import MDAnalysis as mda
from MDAnalysis import transformations
import nglview as nv

import os,re

%run -i _functions.py

## read trajectory

* we created our system by using ```fftool```, the generated ```data.lmp``` can be use to determin the element of different types of atoms in ```lammps```
* there are 70 pairs of cations and anions in the system
* there are 10 frames in the file

In [2]:
directory = './'
filename = directory+'data.lmp'
## elements
elements = fetchList(filename, 'Masses', 0,4, skipBlankSplit=True)
digitspattern = r'#'
types = []
chemical_symbols = []
for element in elements:
    types.append(int(element[0]))
    txt = re.sub(digitspattern, '', element[-1])
    chemical_symbols.append(atomic_symbol(txt))
types = np.array(types)

print('types:            ',types)
print('chemical_symbols: ',chemical_symbols)

types:             [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
chemical_symbols:  ['N', 'C', 'C', 'C', 'H', 'C', 'H', 'H', 'C', 'C', 'H', 'C', 'B', 'F']


In [3]:
il_num=70 # 70 pairs of ions in the system

# full: atom-ID molecule-ID atom-type q x y z
u = mda.Universe('result_atoms.data', atom_style='id resid type charge x y z')

## elements
elements = []
for i in range(len(u.atoms)):
    elements.append(chemical_symbols[np.where(types==int(u.atoms[i].type))[0][0]])
u.add_TopologyAttr('element',values=elements)

## resnames
resnames = ['cati']*il_num+['anio']*il_num
u.add_TopologyAttr('resnames',values=resnames)

u.load_new("result_atoms.lammpstrj", format="LAMMPSDUMP",timeunit="fs",dt=10000)
workflow = [transformations.unwrap(u.atoms)]
u.trajectory.add_transformations(*workflow)

print('dimensions:',u.dimensions)
print('frames',u.trajectory.n_frames)

view = nv.show_mdanalysis(u)
view.clear_representations()
view.add_representation(selection='cati',repr_type='licorice',radius='0.5',opacity=1)
view.add_representation(selection='anio',repr_type='licorice',radius='0.5',opacity=1)
view.add_unitcell()
view.camera = 'orthographic'
view.control.spin([1,0,0],-np.pi/2.)
view.control.spin([0,1,0],-np.pi/2.)
view

dimensions: [ 40.  40. 400.  90.  90.  90.]
frames 10


NGLWidget(max_frame=9)

In [4]:
view.render_image(factor=8,antialias=True,trim=True,transparent=True)

Image(value=b'', width='99%')

<img title="image" alt="image" src="./images/sim_box.png" width="600">

## compute the center-of-mass of cations

In [11]:
cations = u.select_atoms('resname cati') # select all cations
resids = np.unique(cations.atoms.resids) # extract resid of all cations

cation_coms = []
for ts in u.trajectory:          
    for i in resids: 
        sel = u.select_atoms('resid %d'%i)   
        com = sel.center_of_mass()    
        cation_coms.append([com[0],com[1],com[2]])

## compute the normalized vector connecting head and tail of cations

* we choose the center of two N atoms as the coordinate of the head
* we choose the ```type==12``` atoms, which is the last C atom at the alkyl chain, as the coordinate of the tail

In [12]:
cation_oriens = []
for ts in u.trajectory:          
    for i in resids: 
        sel = u.select_atoms('resid %d'%i)   
        head = u.select_atoms('resid %d and element N'%i)   
        tail = u.select_atoms('resid %d and type 12'%i)  
        com = head.center_of_mass()
        vec = tail.center_of_mass()-com
        vec_norm = np.linalg.norm(vec)  
        vec = vec/vec_norm  # normalized
        cation_oriens.append([vec[0],vec[1],vec[2]])

* we compute the $\cos\theta$ between the calculated vector and $z=(1,0,0)$ to show the orientation of cations

In [13]:
z = np.array([1,0,0])
costheta = [np.dot(z,cation_oriens[i]) for i in range(len(cation_oriens))]
costheta = np.array(costheta)

## put all data together

* we create a ```pandas``` dataframe to show the result
* we will have 700 rows of data, representing properties of 70 cations in 10 frames
* the vector $ux, uy, uz$ can be used to compute the orientation of cations respected to a given vector

In [18]:
cation_props = np.hstack([cation_coms,cation_oriens,costheta[:,None]])
cation_props_pd = pd.DataFrame(cation_props,columns=['cmx', 'cmy', 'cmz', 'ux', 'uy', 'uz', 'costheta'])
display(cation_props_pd)

,cmx,cmy,cmz,ux,uy,uz,costheta
0,22.034613,20.057598,193.741971,-0.176142,0.766780,-0.617270,-0.176142
1,9.325602,27.485126,210.081804,-0.438095,-0.788806,0.431112,-0.438095
2,28.243463,5.454893,204.909815,-0.409787,-0.256199,0.875464,-0.409787
...,...,...,...,...,...,...,...
697,10.092617,21.561858,205.189105,0.775252,-0.088744,0.625386,0.775252
698,22.672758,32.295271,196.962807,0.231209,-0.212964,-0.949310,0.231209
699,37.018309,8.695704,203.219415,0.449726,0.230745,0.862846,0.449726


* we can save the results in a file, and then reload again

In [15]:
with open('result_cation_props.dat','w') as f:
    f.write(' '.join(cation_props_pd.columns))
    f.write('\n')
    np.savetxt(f,cation_props_pd.values,header='properties of cations',comments='#')

* reload

In [17]:
filename = 'result_cation_props.dat'
cation_props_pd = pd.read_csv(filename,sep='\s+',comment='#')
display(cation_props_pd)

,cmx,cmy,cmz,ux,uy,uz,costheta
0,22.034613,20.057598,193.741971,-0.176142,0.766780,-0.617270,-0.176142
1,9.325602,27.485126,210.081804,-0.438095,-0.788806,0.431112,-0.438095
2,28.243463,5.454893,204.909815,-0.409787,-0.256199,0.875464,-0.409787
...,...,...,...,...,...,...,...
697,10.092617,21.561858,205.189105,0.775252,-0.088744,0.625386,0.775252
698,22.672758,32.295271,196.962807,0.231209,-0.212964,-0.949310,0.231209
699,37.018309,8.695704,203.219415,0.449726,0.230745,0.862846,0.449726
